In [1]:
import cv2
import mediapipe as mp
import os

class PoseDetector():
    '''
    人体姿势检测类
    '''
    def __init__(self, static_image_mode=False, upper_body_only=False,
                 smooth_landmarks=True, min_detection_confidence=0.5,
                 min_tracking_confidence=0.5):
        '''
        初始化
        :param static_image_mode: 是否是静态图片，默认为否
        :param upper_body_only: 是否是上半身，默认为否
        :param smooth_landmarks: 设置为True减少抖动
        :param min_detection_confidence: 人员检测模型的最小置信度值，默认为0.5
        :param min_tracking_confidence: 姿势可信标记的最小置信度值，默认为0.5
        '''
        self.static_image_mode = static_image_mode
        self.upper_body_only = upper_body_only
        self.smooth_landmarks = smooth_landmarks
        self.min_detection_confidence = min_detection_confidence
        self.min_tracking_confidence = min_tracking_confidence
        
        # 设置模型复杂度（0: 简单模型，1: 复杂模型）
        self.model_complexity = 1  # 选择复杂模型
        
        # 创建一个Pose对象用于检测人体姿势
        self.pose = mp.solutions.pose.Pose(
            static_image_mode=self.static_image_mode,
            model_complexity=self.model_complexity,
            smooth_landmarks=self.smooth_landmarks,
            min_detection_confidence=self.min_detection_confidence,
            min_tracking_confidence=self.min_tracking_confidence
        )

    def find_pose(self, img, draw=True):
        '''
        检测姿势方法
        :param img: 一帧图像
        :param draw: 是否画出人体姿势节点和连接图
        :return: 处理过的图像
        '''
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                mp.solutions.drawing_utils.draw_landmarks(img, self.results.pose_landmarks,
                                                           mp.solutions.pose.POSE_CONNECTIONS)
        return img


# 视频路径和输出路径
video_path = r'C:\data\video\0-两手托天理三焦（八段锦）\standard_0.mp4'
output_path = r'C:\data\result\bone_recognition\movenet'

# 确保输出目录存在
os.makedirs(output_path, exist_ok=True)

In [2]:
# 开始视频处理
cap = cv2.VideoCapture(video_path)

# 获取视频的帧率和大小
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义输出视频的编码和输出文件名
output_file = os.path.join(output_path, 'output_pose_detection.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用mp4v编码
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

detector = PoseDetector()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Error: Could not read frame.")
        break

    # 处理帧并绘制姿势
    img_pose = detector.find_pose(frame)
    
    # 写入输出视频
    out.write(img_pose)

    # 显示图像
    cv2.imshow('Pose Detection', img_pose)

    # 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"输出视频已保存到: {output_file}")

Error: Could not read frame.
输出视频已保存到: C:\data\result\bone_recognition\movenet\output_pose_detection.mp4


In [7]:
import cv2
import mediapipe as mp

# 设置输入视频路径和输出视频路径
input_video = r'C:\data\video\0-两手托天理三焦（八段锦）\standard_0.mp4'
output_video = r'C:\data\result\bone_recognition\movenet\output_pose_detection.mp4'

# 初始化MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 初始化视频捕获
cap = cv2.VideoCapture(input_video)

# 获取视频信息
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 设置视频输出
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

# 定义关键点连接关系
connections = [
    (mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_EYE),
    (mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.RIGHT_EYE),
    (mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EAR),
    (mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.RIGHT_EAR),
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
    (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
    (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
    (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
    (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
    (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
    (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
    (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
    (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE)
]

# 处理视频帧
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Warning: 无法读取帧，退出...")
        break

    # 转换图像为RGB格式
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 处理图像并获取结果
    results = pose.process(image_rgb)

    # 绘制关键点和连接线
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # 绘制自定义连接
        for connection in connections:
            start = results.pose_landmarks.landmark[connection[0]]
            end = results.pose_landmarks.landmark[connection[1]]
            start_point = (int(start.x * frame_width), int(start.y * frame_height))
            end_point = (int(end.x * frame_width), int(end.y * frame_height))
            cv2.line(frame, start_point, end_point, (0, 255, 0), 2)  # 绿色连接线

    # 写入输出视频
    out.write(frame)

    # 显示结果
    cv2.imshow('Pose Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"输出视频已保存到: {output_video}")


输出视频已保存到: C:\data\result\bone_recognition\movenet\output_pose_detection.mp4
